In [89]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from plotly.offline import init_notebook_mode, iplot

dataset = pd.read_csv(r'/home/sandaruwan/rest_api/rest_api/datasets/foods_details.csv')

dataset = dataset.drop_duplicates()
dataset.head()


,name,carbs,fiber,sugar,fat,protien,glycamic_index,meal_type,meal_for,image_link,Unnamed: 10
0,Almond Butter and Celery Sticks,5,3,1,7.0,2.0,0,snack,veg,http://drive.google.com/uc?export=view&id=1m6T...,NaN
1,Almonds,5,3,1,14.0,6.0,0,snack,veg,http://drive.google.com/uc?export=view&id=1OuQ...,NaN
2,Apple,25,4,19,0.5,0.5,53,snack,veg,http://drive.google.com/uc?export=view&id=1qaS...,NaN
3,Avocado,2,2,1,15.0,2.0,15,snack,veg,http://drive.google.com/uc?export=view&id=1dqN...,
4,Avocado and Tomato Salad,10,5,2,15.0,3.0,20,lunch,veg,http://drive.google.com/uc?export=view&id=15iu...,


In [67]:
ststus = "non-veg"
index = 140 - 100
features = ['glycamic_index']

dataset['combined_features'] = dataset[features].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
dataset_condition = dataset[dataset['meal_for'] == ststus][dataset['glycamic_index'] >= index]

tfidf = TfidfVectorizer(stop_words='english')

tfdif_matrix = tfidf.fit_transform(dataset_condition['combined_features'])

cosine_sim = cosine_similarity(tfdif_matrix, tfdif_matrix)

recomendations = np.argsort(cosine_sim[-1])[:20-0]

for rec in recomendations[1:]:
    print('- {}'.format(dataset_condition.iloc[rec]['name']))

dataset.reset_index(inplace=True)





- Grilled Chicken Caesar Salad
- Salmon and Asparagus Foil Packets
- Salmon Salad
- Tuna Salad
- Bison Burger with Lettuce Wrap
- Spinach Salad with Grilled Shrimp
- Whole Wheat Pasta with Tomato Sauce


/tmp/ipykernel_39832/1848975726.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [92]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import random

diabetics_score = 100
veg_status = "veg"

dataset = pd.read_csv(r"/home/sandaruwan/rest_api/rest_api/datasets/foods_details.csv")

dataset["diabetic_suitability"] = dataset["glycamic_index"].apply(lambda x: 0 if diabetics_score > 100 and x > 45 else 1 )
dataset.head()

features = ["carbs", "fiber", "sugar", "fat", "protien", "glycamic_index"]

X_train, X_test, Y_train, Y_test = train_test_split(dataset[features],dataset['diabetic_suitability'],test_size=0.1, random_state=random.randint(1,200), shuffle=True)

model = RandomForestClassifier(n_estimators=100, random_state=102)
model.fit(X_train,Y_train)

predicted_lables = model.predict(X_test)
predicted_array = np.array(predicted_lables)


recomended_data = pd.concat([X_test], axis=1)

if 'diabetic_suitability' in dataset.columns:
  recomended_data = pd.concat([recomended_data, dataset[['diabetic_suitability']]], axis=1)

print(recomended_data.head())

    carbs  fiber  sugar   fat  protien  glycamic_index  diabetic_suitability
29   20.0    1.0   15.0   0.0      1.0            59.0                     1
53    0.0    0.0    0.0   6.0     20.0             5.0                     1
47   20.0    3.0   15.0   0.0      1.0            45.0                     1
37    5.0    3.0    2.0   0.0      1.0            15.0                     1
3     2.0    2.0    1.0  15.0      2.0            15.0                     1


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import random

# Assuming the CSV
dataset = pd.read_csv("/home/sandaruwan/rest_api/rest_api/datasets/foods_details.csv")
diabetics_score = 100
# Function to filter vegetarian/non-vegetarian foods (adapt based on your data)
def filter_veg_status(data, veg_status):
    if veg_status == "veg":
        return data[data["meal_for"] == "veg"]  # Assuming a "veg" column exists
    elif veg_status == "non-veg":
        return data[data["meal_for"] == "non-veg"]
    else:
        return data  # Return all data if veg_status is not specified

# Filter dataset based on veg_status
filtered_data = filter_veg_status(dataset.copy(), "veg")

# Create diabetic suitability label considering diabetics_score
filtered_data["diabetic_suitability"] = filtered_data["glycamic_index"].apply(lambda x: 0 if diabetics_score > 100 and x > 45 else 1)

# Select features and split data
features = ["carbs", "fiber", "sugar", "fat", "protien", "glycamic_index"]
X_train, X_test, Y_train, Y_test = train_test_split(
    filtered_data[features],
    filtered_data["diabetic_suitability"],
    test_size=0.1,
    random_state=random.randint(1, 200),
    shuffle=True,
)

# Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Make predictions and create recommended data
predicted_labels = model.predict(X_test)
recommended_data = pd.concat([X_test, pd.DataFrame(predicted_labels, columns=["diabetic_suitability"])], axis=1).head(2)

# Print the first few rows of recommended data with diabetic suitability

# Increase data size (consider data availability and computational resources)
if len(dataset) > len(filtered_data):
    more_data = dataset.sample(len(filtered_data))  # Sample additional data from original dataset
    filtered_data = pd.concat([filtered_data, more_data], ignore_index=True)
    # Repeat training and prediction steps using the expanded filtered_data
filtered_data.head(2)

,name,carbs,fiber,sugar,fat,protien,glycamic_index,meal_type,meal_for,image_link,Unnamed: 10,diabetic_suitability
0,Almond Butter and Celery Sticks,5,3,1,7.0,2.0,0,snack,veg,http://drive.google.com/uc?export=view&id=1m6T...,NaN,1.0
1,Almonds,5,3,1,14.0,6.0,0,snack,veg,http://drive.google.com/uc?export=view&id=1OuQ...,NaN,1.0


In [117]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import random

# read dataset
dataset = pd.read_csv(r"/home/sandaruwan/rest_api/rest_api/datasets/meal_plans.csv")

# filter data veg, non-veg status
def filter_veg_status(data, status):
    if status == "veg":
        return data[data['preference_type'] == "veg"]
    else :
        return data
    
# filter data by level
def filter_by_diabetics_level(data, status):
    if status == "low":
        return data[data['diabetics_level'] == "low"]
    elif status == "high":
        return data[data['diabetics_level'] == "high"]
    else:
        return data
    
filtered_data1 = filter_veg_status(dataset, "veg")
filtterd_data2 = filter_by_diabetics_level(filtered_data1, "low")


features = ["carb", "proteins", "fat", "glycemic_index"]
target = "diabetics_level"

X_train, X_test, Y_train, Y_test = train_test_split(filtterd_data2[features], filtterd_data2[target], test_size=0.8)

model = RandomForestClassifier(n_estimators=100, random_state=5)
model.fit(X=X_train, y=Y_train)

predicted_lables = model.predict(X_test)
recomend_data = pd.concat([X_test, pd.DataFrame(predicted_labels, columns=["carb"])], axis=1)

final_data = filtterd_data2.sample(frac=1)
final_data.head()




,meal_type,preference_type,meals_name,cal,carb,proteins,fat,recipe,glycemic_index,diabetics_level,image_link
16,Snack,veg,Whole Milk Ice Cream,273,5,31,15,1 cup regular whole milk ice cream,44,low,NaN
5,Snack,veg,Banana and Almond Butter,302,33,8,18,"1 banana,2 tablespoons almond butter",40,low,NaN
7,Snack,veg,Greek Yogurt with Berries,180,20,10,5,"1 cup 2% Greek yogurt,Mixed berries",55,low,NaN
0,Breakfast,veg,Greek Yogurt Parfait,313,25,27,13,"1 cup 2% plain Greek yogurt,1/2-ounce chopped ...",45,low,NaN
15,Dinner,veg,Tofu Teriyaki with Brown Rice and Broccoli,491,58,20,21,"4 ounces tofu,1 cup brown rice,1/2 cup broccol...",46,low,NaN
